In [119]:
!pip install transformers

In [120]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
#import plotly.express as px
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
import string
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
import numpy as np

In [121]:
train_df = pd.read_csv('train.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [122]:
test_df = pd.read_csv('test.csv')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [123]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [124]:
#limipeza
#import nltk
#nltk.download('stopwords')

def clean(text):
    
    exclude = set(string.punctuation)
    clean_text = ''.join(ch for ch in text if ch not in exclude)
    
    exclude = set(stopwords.words('english'))
    clean_text_list = clean_text.split(' ')
    clean_text = ' '.join(ch for ch in clean_text_list if ch not in exclude)

    return clean_text

test_df['clean_text'] = test_df['text'].apply(clean)
train_df['clean_text'] = train_df['text'].apply(clean)
train_df.head()

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds Reason earthquake May ALLAH Forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked shelter place notified off...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent photo Ruby Alaska smoke wildfire...


In [125]:
def trp(l, n):
    return l[:n] + [0]*(n-len(l))


In [126]:
def tokenizacion_de_texto(texto,longitud):
    marked_text = "[CLS] " + texto + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    vector = trp(indexed_tokens,longitud)
    return vector
    #return indexed_tokens

In [127]:
train_df = train_df.fillna('')
test_df = test_df.fillna('')

In [128]:

train_df['texto_tokenizado'] = train_df['clean_text'].apply(lambda x: tokenizacion_de_texto(x,80))
train_df['keyword_tokenizado'] = train_df['keyword'].apply(lambda x: tokenizacion_de_texto(x,5))
train_df['location_tokenizado'] = train_df['location'].apply(lambda x: tokenizacion_de_texto(x,7))
train_df['features_tokenizados'] = (train_df['texto_tokenizado'] + train_df['keyword_tokenizado'] + train_df['location_tokenizado'])
#XTrain = np.array(train_df['features_tokenizados'])

test_df['texto_tokenizado'] = test_df['clean_text'].apply(lambda x: tokenizacion_de_texto(x,80))
test_df['keyword_tokenizado'] = test_df['keyword'].apply(lambda x: tokenizacion_de_texto(x,5))
test_df['location_tokenizado'] = test_df['location'].apply(lambda x: tokenizacion_de_texto(x,7))
test_df['features_tokenizados'] = (test_df['texto_tokenizado'] + test_df['keyword_tokenizado'] + test_df['location_tokenizado'])
#XTest = hstack(np.array(test_df['texto_tokenizado']),np.array(test_df['keyword_tokenizado']),np.array(test_df['location_tokenizado']))




In [129]:
train_df.head()

,id,keyword,location,text,target,clean_text,texto_tokenizado,keyword_tokenizado,location_tokenizado,features_tokenizados
0,1,,,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds Reason earthquake May ALLAH Forgive us,"[101, 2256, 15616, 3114, 8372, 2089, 16455, 96...","[101, 102, 0, 0, 0]","[101, 102, 0, 0, 0, 0, 0]","[101, 2256, 15616, 3114, 8372, 2089, 16455, 96..."
1,4,,,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada,"[101, 3224, 2543, 2379, 2474, 6902, 3351, 2187...","[101, 102, 0, 0, 0]","[101, 102, 0, 0, 0, 0, 0]","[101, 3224, 2543, 2379, 2474, 6902, 3351, 2187..."
2,5,,,All residents asked to 'shelter in place' are ...,1,All residents asked shelter place notified off...,"[101, 2035, 3901, 2356, 7713, 2173, 19488, 373...","[101, 102, 0, 0, 0]","[101, 102, 0, 0, 0, 0, 0]","[101, 2035, 3901, 2356, 7713, 2173, 19488, 373..."
3,6,,,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...,"[101, 19527, 2692, 2111, 4374, 3748, 26332, 13...","[101, 102, 0, 0, 0]","[101, 102, 0, 0, 0, 0, 0]","[101, 19527, 2692, 2111, 4374, 3748, 26332, 13..."
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent photo Ruby Alaska smoke wildfire...,"[101, 2074, 2288, 2741, 6302, 10090, 7397, 561...","[101, 102, 0, 0, 0]","[101, 102, 0, 0, 0, 0, 0]","[101, 2074, 2288, 2741, 6302, 10090, 7397, 561..."


In [130]:
#split
X_train, X_test, y_train, y_test = train_test_split(train_df['features_tokenizados'], train_df['target'], test_size=0.33,
                                                  random_state=42)
FTest = test_df['features_tokenizados']
test_ids = test_df['id'] 

In [131]:
#Parametros
vocab_size = 30522
epochs = 2
maxlen = 92
n_words = 500
embedding_dim = 16

def tokenizar(textos):
    secuencia = []
    for text in textos:
        marked_text = "[CLS] " + text + " [SEP]"
        tokenized_text = tokenizer.tokenize(marked_text)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        secuencia.appFailed to convert a NumPy array to a Tensor (Unsupported object type list).end(indexed_tokens)
    return secuencia


In [132]:
X_train = np.array([np.array(lista) for lista in X_train])
X_test = np.array([np.array(lista) for lista in X_test])
padded_FTest = X_test = np.array([np.array(lista) for lista in FTest])
padded_train = pad_sequences(X_train, maxlen = maxlen, truncating = 'post')
padded_test = pad_sequences(X_test, maxlen = maxlen, truncating = 'post')

type(X_train[0])

numpy.ndarray

In [133]:
#secuencia_train = tokenizar(X_train.values)
#secuencia_test = tokenizar(X_test.values)
#secuencia_FTest = tokenizar(FTest.values)

#padded_train = pad_sequences(secuencia_train, maxlen = maxlen, truncating = 'post')
#padded_test = pad_sequences(secuencia_test, maxlen = maxlen)
#padded_FTest = pad_sequences(secuencia_FTest, maxlen = maxlen)

In [134]:
#(padded_FTest[0])

In [135]:
#len(secuencia_FTest[125])

In [136]:
#modelo Conv1D
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length= maxlen),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 92, 64)            1953408   
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 88, 128)           41088     
_________________________________________________________________
global_average_pooling1d_10  (None, 128)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 65        
Total params: 2,002,817
Trainable params: 2,002,817
Non-trainable params: 0
_________________________________________________________________


In [137]:
model.fit(padded_train, y_train, epochs=epochs, validation_data=(padded_test, y_test))

Epoch 1/2
160/160 [==============================] - ETA: 0s - loss: 0.6191 - accuracy: 0.6471

ValueError: Data cardinality is ambiguous:
  x sizes: 3263
  y sizes: 2513
Please provide data which shares the same first dimension.

preds = model.predict_classes(padded_test)
#preds = pd.Series(list((x[0] for x in preds)))

print("Score RF:", f1_score(padded_FTest, preds))
#preds = pd.Series(preds)


In [ ]:
preds = pd.Series(list((x[0] for x in preds)))
preds
df_preds = pd.concat([test_ids,preds],axis=1)
df_preds.rename(columns = {0 : 'target'}, inplace=True)
df_preds.set_index('id', inplace=True)
df_preds.to_csv('BERT-Conv1D.csv')
df_preds.head()

Tuning: 
1-  epochs = 10
    maxlen = 50
    n_words = 500
    embedding_dim = 16
Acuracy: 0.7441

2-  epochs = 6
    maxlen = 40
    n_words = 500
    embedding_dim = 16
Accuracy : 0.7656b

3-  epochs = 5
    maxlen = 40
    n_words = 300
    embedding_dim = 16
Accuracy: 0.7437
s